<a href="https://colab.research.google.com/github/Ofiregev/Final_Project-FetalCns/blob/main/ofir's_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import pandas as pd
import torch.nn.functional as F
from sklearn.model_selection import KFold
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

print("Part 1 has done")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Part 1 has done


In [2]:
from sklearn.preprocessing import StandardScaler

class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

        # Initialize StandardScaler for normalization
        self.scaler = StandardScaler()
        self.labels = self.data_frame.iloc[:, 2].values.reshape(-1, 1)
        self.labels = self.scaler.fit_transform(self.labels)

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)

        # Get the normalized label
        label = self.labels[idx]

        return image, label


In [3]:

# Determine the available hardware (CPU or GPU) and set the PyTorch device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [4]:
# Read the list of images from the CSV file
train_csv = "/content/drive/MyDrive/FinalProject/training_set_pixel_size_and_HC.csv"
train_csv_df = pd.read_csv(train_csv)


In [5]:
train_csv_df.head()
len(train_csv_df)

999

In [6]:
# Part 3: Define Transforms and Create Dataset

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
])
train_dataset = CustomDataset(csv_file="/content/drive/MyDrive/FinalProject/training_set_pixel_size_and_HC.csv",
                              root_dir="/content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/",
                              transform=transform)

In [7]:
# Part 4: Create DataLoader
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [8]:
# Part 5: Define Model
class Model(nn.Module):
    def __init__(self, input_channels=1, input_size=64):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv4 = nn.Conv2d(256, 512, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        conv_output_size = input_size // 2 // 2 // 2 // 2  # Adjusted for additional pooling layers
        self.fc1 = nn.Linear(512 * conv_output_size * conv_output_size, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)
        self.dropout = nn.Dropout(0.3)
        self.batchnorm1 = nn.BatchNorm2d(64)
        self.batchnorm2 = nn.BatchNorm2d(128)
        self.batchnorm3 = nn.BatchNorm2d(256)
        self.batchnorm4 = nn.BatchNorm2d(512)

    def forward(self, x):
        x = self.pool(F.relu(self.batchnorm1(self.conv1(x))))
        x = self.pool(F.relu(self.batchnorm2(self.conv2(x))))
        x = self.pool(F.relu(self.batchnorm3(self.conv3(x))))
        x = self.pool(F.relu(self.batchnorm4(self.conv4(x))))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

In [9]:
model = Model(input_channels=1, input_size=64)
model.to(device)

Model(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8192, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (batchnorm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4)

In [10]:
# Part 6: Define Training Parameters and Optimizer
learning_rate = 0.001
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)


In [11]:
def train(model, train_loader, optimizer, criterion, epochs, device):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs = inputs.float().to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")

In [12]:
def evaluate(model, val_loader, criterion, device):
    model.eval()
    val_losses = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.float().to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(inputs)
            val_loss = criterion(outputs, labels)
            val_losses.append(val_loss.item())

    fold_avg_val_loss = np.mean(val_losses)
    return fold_avg_val_loss

In [13]:
k = 3
kf = KFold(n_splits=k, shuffle=True, random_state=42)
evaluation_metrics = []
epochs = 8

for fold_idx, (train_idx, val_idx) in enumerate(kf.split(train_dataset)):
    print(f"Fold {fold_idx + 1}")
    print("=========")

    train_data_fold = [train_dataset[i] for i in train_idx]
    val_data_fold = [train_dataset[i] for i in val_idx]

    train_loader_fold = DataLoader(train_data_fold, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader_fold = DataLoader(val_data_fold, batch_size=batch_size, num_workers=4)

    train(model, train_loader_fold, optimizer, criterion, epochs, device)
    fold_avg_val_loss = evaluate(model, val_loader_fold, criterion, device)
    evaluation_metrics.append(fold_avg_val_loss)

    print(f"Fold {fold_idx + 1} Validation Loss: {fold_avg_val_loss:.4f}")
    print("=========")

average_metric = np.mean(evaluation_metrics)
print("Average Evaluation Metric:", average_metric)

Fold 1


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([8, 1, 1])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  ret

Epoch 1, Loss: 1.6999
Epoch 2, Loss: 1.0179
Epoch 3, Loss: 1.0164
Epoch 4, Loss: 1.0158
Epoch 5, Loss: 1.0294
Epoch 6, Loss: 1.0271
Epoch 7, Loss: 1.0124
Epoch 8, Loss: 1.0295


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([5, 1, 1])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold 1 Validation Loss: 1.0085
Fold 2
Epoch 1, Loss: 0.9465
Epoch 2, Loss: 0.9382
Epoch 3, Loss: 0.9545
Epoch 4, Loss: 0.9509
Epoch 5, Loss: 0.9646
Epoch 6, Loss: 0.9480
Epoch 7, Loss: 0.9536
Epoch 8, Loss: 0.9452
Fold 2 Validation Loss: 1.1481
Fold 3
Epoch 1, Loss: 1.0370
Epoch 2, Loss: 1.0729
Epoch 3, Loss: 1.0482
Epoch 4, Loss: 1.0325
Epoch 5, Loss: 1.0339
Epoch 6, Loss: 1.0367
Epoch 7, Loss: 1.0326
Epoch 8, Loss: 1.0588
Fold 3 Validation Loss: 0.9562
Average Evaluation Metric: 1.037597365390938


In [14]:
# Part 8: Save the trained model
model_path = "/content/drive/MyDrive/FinalProject/trained_model.pth"
torch.save(model.state_dict(), model_path)
print("Model saved successfully!")

Model saved successfully!


In [15]:
# # Load the trained model
model_path = "/content/drive/MyDrive/FinalProject/trained_model.pth"
model = Model(input_channels=1, input_size=64)
model.load_state_dict(torch.load(model_path))
model.eval()

Model(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8192, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (batchnorm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4)

In [34]:
def inverse_normalize(predicted, normalization_params):
    """
    Apply inverse normalization to predicted values.

    Args:
    - predicted: Predicted values after normalization
    - normalization_params: Parameters used for normalization (e.g., mean, std)

    Returns:
    - Predicted values in the original scale
    """
    # Apply inverse normalization based on the normalization parameters
    # For example, if mean and std were used for normalization:
    mean = normalization_params['mean']
    std = normalization_params['std']
    predicted_original_scale = predicted * std + mean

    return predicted_original_scale.item()


In [24]:
scaler =train_dataset.scaler
# Store mean and standard deviation in normalization_params
normalization_params = {
    'mean': scaler.mean_,
    'std': scaler.scale_
}

In [35]:
def predict_head_circumference(image_path, model, normalization_params):
    # Open and preprocess the new image
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor()
    ])
    image = Image.open(image_path)
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

    # Make prediction
    with torch.no_grad():
        output = model(image_tensor.float())

    # Apply inverse normalization
    predicted_circumference_normalized = output.item()
    predicted_circumference_original_scale = inverse_normalize(predicted_circumference_normalized, normalization_params)

    return predicted_circumference_original_scale  # Return the predicted head circumference in the original scale


In [36]:
# Choose 10 random indices from the dataset
num_images_to_test = 500
random_indices = np.random.choice(len(train_csv_df), size=num_images_to_test, replace=False)
# Initialize a list to store the absolute errors
absolute_errors = []

# Loop through the selected images
for idx in random_indices:
    image_path = os.path.join("/content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/", train_csv_df.iloc[idx, 0])
    ground_truth_circumference = train_csv_df.iloc[idx, 2]  # Assuming the third column contains the head circumference labels

    # Call the function to predict head circumference from the image
    predicted_circumference = predict_head_circumference(image_path, model, normalization_params)

    # Calculate the absolute error and add it to the list
    absolute_error = abs(predicted_circumference - ground_truth_circumference)
    absolute_errors.append(absolute_error)

    # Print the predicted and ground truth head circumferences
    print("Image:", image_path)
    print("Predicted Head Circumference:", predicted_circumference)
    print("Ground Truth Head Circumference:", ground_truth_circumference)
    print("Absolute Error:", absolute_error)
    print("==============================================")

# Calculate the average absolute error
average_absolute_error = sum(absolute_errors) / len(absolute_errors)
print("Average Absolute Error:", average_absolute_error)


Image: /content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/490_2HC.png
Predicted Head Circumference: 175.5708197358634
Ground Truth Head Circumference: 191.9
Absolute Error: 16.329180264136596
Image: /content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/776_HC.png
Predicted Head Circumference: 175.4135888950065
Ground Truth Head Circumference: 289.2
Absolute Error: 113.78641110499348
Image: /content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/657_2HC.png
Predicted Head Circumference: 175.41418345126792
Ground Truth Head Circumference: 231.2
Absolute Error: 55.78581654873207
Image: /content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/532_2HC.png
Predicted Head Circumference: 175.3994536601939
Ground Truth Head Circumference: 185.12
Absolute Error: 9.720546339806106
Image: /content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/009_HC.png
Predicted Head Circumference: 174.81529988494998
Ground Truth Head Ci

In [ ]:
# # Path for the new image
# new_image_path = "/content/drive/MyDrive/FinalProject/Dataset/training_set/training_set/001_HC.png"
# # Call the function to predict head circumference from the new image
# predicted_circumference = predict_head_circumference(new_image_path, model)
# print("Predicted Head Circumference:", predicted_circumference)
